# 토픽모델링을 통한 키워드 분석

## **현재 코드를 제대로 활용하시려면 Chrome으로 실행시켜주세요! **

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 수집한 데이터들중 원하는 가져오기 위해 키워드를 입력합니다.
query_keyword = input("분석을 위한 키워드를 입력하세요 : ")
start, end, step = [int(x) for x in input("원하는 토픽 갯수들을 입력하세요(e.g. 2,5,1)").split(",")]

분석을 위한 키워드를 입력하세요 : 문화
원하는 토픽 갯수들을 입력하세요(e.g. 2,5,1)2,10,1


In [ ]:
import pickle
base_path = "/content/drive/MyDrive/Colab Notebooks/dataset/"

with open(base_path + f"tokenized_docs({query_keyword}).pk", "rb") as f:
    tokenized_docs = pickle.load(f)

In [ ]:
!pip install pyldavis==3.2.2

In [ ]:
import sys
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm_notebook # progress bar

import numpy as np
import pandas as pd
import string
import re
import warnings
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
%matplotlib inline

## 운영체제별 글꼴 세팅
# 그래프를 이쁘게 그리기 위한 코드입니다. 한글 글꼴을 추가합니다.

%matplotlib inline  

import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
import seaborn as sns
mpl.rcParams['axes.unicode_minus'] = False

sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져옵니다.
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

# 현재 설정되어 있는 폰트 사이즈와 글꼴을 알아보자
!python --version
def current_font():
  print(f"설정 폰트 글꼴: {plt.rcParams['font.family']}, 설정 폰트 사이즈: {plt.rcParams['font.size']}")  # 파이썬 3.6 이상 사용가능하다
        
current_font()

In [ ]:
# 여전히 글꼴이 보이지 않는 분들은, 런타임 -> "다시 시작 및 모두 실행" 을 눌러주세요!
fm._rebuild()

### gensim LDA model을 사용하기 위한 자료구조 생성. 

In [ ]:
# 토픽모델링을 위한 라이브러리 불러오기
from gensim import corpora
from gensim import models

In [ ]:
# 문서-단어 행렬 만들기 # LDA의 input.
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_docs)
# 문서-단어 행렬 생성
corpus = [dictionary.doc2bow(document) for document in tokenized_docs]

In [ ]:
NUM_TOTAL_WORDS = len(dictionary)
print(dictionary)
print(NUM_TOTAL_WORDS)

## 3. 토픽 모델링(LDA Model)

In [ ]:
# gensim에서 ldamodel 불러오기
from gensim.models import ldamodel
# lda를 측정하는 지표인 coherence score 계산
from gensim.models import CoherenceModel

from time import time
import os
import gensim

In [ ]:
# 토픽 개수와 토픽 별 상위 추출 단어 개수 지정.
NUM_TOPIC_WORDS = 15


def build_doc_term_mat(documents):
    """주어진 문서 집합으로 문서-어휘 행렬을 만들어 돌려준다."""
    
    print_log_msg("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]

    return corpus, dictionary


def print_topic_words(model): # model <- (학습이 완료된)ldamodel
    """토픽별 토픽 단어들을 화면에 인쇄한다."""
    
    print_log_msg("Printing topic words.")
    
    
    for topic_id in range(model.num_topics):
        #model.show_topic(0, 15) -> 0번 토픽에서 상위 15개의 단어를 출력해라.
        # -> (단어, 확률)을 원소로 하는 리스트 반환.
        #model.show_topic(1, 30) -> 1번 토픽에서 상위 30개의 단어를 출력해라.
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))

        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))

        print("\n")


def print_log_msg(msg):
    """로그 메시지를 출력한다."""
    
    print(msg, flush=True)

In [ ]:
def compute_coherence(dictionary, corpus, texts, start=8, end=81, step=4):
    coherence_score_list = []
    model_list = []
    # 토픽개수를 2부터 10까지 하고 싶다. -> range(2, 11, 1)
    for num_topics in tqdm_notebook(range(start, end, step)):
        # LDA model 학습코드
        
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, # document-term matrix
                                               num_topics=num_topics, # 토픽개수(K)
                                               id2word=dictionary,
                                               passes=30)
        
        model_list.append(model)
        
        # Coherence Score 학습 코드
        coherence_model = CoherenceModel(model=model,
                                        texts=texts,
                                        dictionary=dictionary,
                                        coherence='c_v')
        coherence_score_list.append(coherence_model.get_coherence())
        
    return model_list, coherence_score_list

In [ ]:
corpus, dictionary = build_doc_term_mat(tokenized_docs)
print(len(corpus), len(dictionary))


# 주어진 토픽 개수들에 대해 LDA model, Coherence model 학습
model_list, coherence_scores = compute_coherence(dictionary=dictionary,
                                                corpus=corpus,
                                                texts=tokenized_docs,
                                                start=start,
                                                end=end,
                                                step=step)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


coherence_list = coherence_scores
label = "Coherenece Score(C_V)"

x = range(start, end, step)
plt.figure(figsize=(16, 12))
plt.xticks(x)
plt.plot(x, coherence_list, label=label)
plt.scatter(x, coherence_list)
plt.title(f"Coherence Scores for LDA with KCI Paper")
plt.xlabel("Num Topics")
plt.ylabel("Coherence Score")
plt.legend(loc='best')
plt.show()

In [ ]:
# 학습한 LDA 모델중에 가장 coherence score가 높은 모델을 선정
# model_list, coherence_list는 여러개의 토픽에 대해서 학습한 모델과 cs를 저장하고 있는 리스트.
# np.argmax는 리스트의 최대값을 가지는 원소의 index를 return.
# e.g. start,end,step = (2,5,1) -> 2,3,4  -> [0, 1, 2]
# 만약에 K=4일 때 베스트라면, np.argmax는 2.
selected_model = model_list[np.argmax(coherence_list)]
selected_model.num_topics

### pyLDAvis를 이용한 시각화 

In [ ]:
# pyLDAvis 불러오기
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(selected_model, corpus, dictionary)
data